In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gc

from sklearn.metrics import mean_absolute_error
import lightgbm as lgbm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# trainの中身をcsv, pickleにしてくれている親切なデータセットがあったので使います
base_dir = '../input/mlb-player-digital-engagement-forecasting/'
train_dir = '../input/mlb-pdef-train-dataset/'

In [ ]:
# このtarget1～4を予測するのがコンペの目的
target_df = pd.read_pickle(train_dir + 'nextDayPlayerEngagement_train.pkl')
target_df.head()

In [ ]:
# target encoding以外の特徴量はこのnotebookを参照
# https://www.kaggle.com/mlconsult/1-38-lb-lightgbm-with-target-statistics

players = pd.read_csv(base_dir + 'players.csv')
rosters = pd.read_pickle(train_dir + 'rosters_train.pkl')
scores = pd.read_pickle(train_dir + 'playerBoxScores_train.pkl')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()

In [ ]:
# 効きそうなカラムを抽出
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances', 'date']

target_stat_cols = ['playerId', 'target1_mean', 'target1_median', 'target1_std', 'target1_max', 'target1_min', 
                       'target2_mean', 'target2_median', 'target2_std', 'target2_max', 'target2_min', 
                       'target3_mean', 'target3_median', 'target3_std', 'target3_max', 'target3_min', 
                       'target4_mean', 'target4_median', 'target4_std', 'target4_max', 'target4_min']

feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
                'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
                'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
                'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
                'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
                'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
                'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
                'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
                'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
                'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
                'groundOutsPitching', 'runsPitching', 'doublesPitching',
                'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
                'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
                'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
                'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
                'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
                'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
                'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
                'inheritedRunnersScored', 'catchersInterferencePitching',
                'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
                'assists', 'putOuts', 'errors', 'chances', 
                'target1_mean', 'target1_median', 'target1_std', 'target1_max', 'target1_min', 
                'target2_mean', 'target2_median', 'target2_std', 'target2_max', 'target2_min', 
                'target3_mean', 'target3_median', 'target3_std', 'target3_max', 'target3_min', 
                'target4_mean', 'target4_median', 'target4_std', 'target4_max', 'target4_min']

In [ ]:
# 結合
df = target_df[targets_cols]

df = df.merge(players[players_cols], on=['playerId'], how='left')
df = df.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
df = df.merge(scores[scores_cols], on=['playerId', 'date'], how='left')

# label encoding
player2num = {c: i for i, c in enumerate(df['playerId'].unique())}
position2num = {c: i for i, c in enumerate(df['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(df['teamId'].unique())}
status2num = {c: i for i, c in enumerate(df['status'].unique())}
df['label_playerId'] = df['playerId'].map(player2num)
df['label_primaryPositionName'] = df['primaryPositionName'].map(position2num)
df['label_teamId'] = df['teamId'].map(teamid2num)
df['label_status'] = df['status'].map(status2num)


del rosters, scores
gc.collect()

df.head()

In [ ]:
with open('player2num.pkl', 'wb') as f:
    pickle.dump(player2num, f)

with open('position2num.pkl', 'wb') as f:
    pickle.dump(position2num, f)

with open('teamid2num.pkl', 'wb') as f:
    pickle.dump(teamid2num, f)

with open('status2num.pkl', 'wb') as f:
    pickle.dump(status2num, f)
    
del player2num, position2num, teamid2num, status2num

In [ ]:
# target encoding用データセット作成
target_stat_df = df.loc[:, ['playerId', 'target1', 'target2', 'target3', 'target4']].groupby('playerId').agg(['mean', 'median', 'std', 'max', 'min'])

In [ ]:
target_stat_df

In [ ]:
target_stat_df = target_stat_df.reset_index()
target_stat_df.columns = target_stat_cols
target_stat_df.head()

In [ ]:
target_stat_df.to_pickle('target_stat_df.pkl')

In [ ]:
df = df.merge(target_stat_df[target_stat_cols], how='left', on='playerId')

del target_stat_df
gc.collect()

df.head()

In [ ]:
df_X = df[feature_cols]
df_y = df[['target1', 'target2', 'target3', 'target4']]

# 全期間を使ってみる
# _index = (df['date'] < 20210401)
x_train = df_X
y_train = df_y
x_valid = df_X
y_valid = df_y

In [ ]:
print('training data shape:' , x_train.shape, y_train.shape)
print('validation data shape:' , x_valid.shape, y_valid.shape)

In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool

def fit_cat(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):

    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = CatBoostRegressor(
                n_estimators=2000,
                learning_rate=0.05,
                loss_function='MAE',
                eval_metric='MAE',
                max_bin=50,
                subsample=0.9,
                colsample_bylevel=0.5,
                verbose=100)
    model.fit(x_train, y_train, use_best_model=True,
                         eval_set=(x_valid, y_valid),
                         early_stopping_rounds=25)
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score

# training xgbm
params = {
'boosting_type': 'gbdt',
'objective':'mae',
'subsample': 0.5,
'subsample_freq': 1,
'learning_rate': 0.03,
'num_leaves': 2**11-1,
'min_data_in_leaf': 2**12-1,
'feature_fraction': 0.5,
'max_bin': 100,
'n_estimators': 2500,
'boost_from_average': False,
"random_seed":42,
    }

oof1, model1_cat, score1 = fit_cat(
    x_train, y_train['target1'],
    x_valid, y_valid['target1'],
    params
    )
oof2, model2_cat, score2 = fit_cat(
    x_train, y_train['target2'],
    x_valid, y_valid['target2'],
    params
    )
oof3, model3_cat, score3 = fit_cat(
    x_train, y_train['target3'],
    x_valid, y_valid['target3'],
    params
    )
oof4, model4_cat, score4 = fit_cat(
    x_train, y_train['target4'],
    x_valid, y_valid['target4'],
    params
    )

score = (score1+score2+score3+score4) / 4
print(f'score: {score}')
    

In [ ]:
pickle.dump(model1_cat, open('model1_cat_all.pkl', 'wb'))
pickle.dump(model2_cat, open('model2_cat_all.pkl', 'wb'))
pickle.dump(model3_cat, open('model3_cat_all.pkl', 'wb'))
pickle.dump(model4_cat, open('model4_cat_all.pkl', 'wb'))